In [1]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
import warnings
warnings.filterwarnings('ignore')
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, classification_report, average_precision_score, accuracy_score
import csv

In [2]:
def get_algo_path():
    import os
    cwd = os.getcwd()
    s='\\'
    if '\\' not in cwd:
        s='/'
    cwd= cwd.split(s)[4:-1]
    cwd.append('Not Normalized')
    cwd='/'.join(cwd)
    return cwd

def get_csv_path():
    import os
    cwd = os.getcwd()
    s="\\"
    if "\\" not in cwd:
        s='/'
    file= cwd.split(s)[:4]
    file.append('models_scores.csv')
    file= s.join(file)
    return file

def line_is_exist(file, row):
    logfile = open(file, 'r')
    loglist = logfile.readlines()
    logfile.close()
    for line in loglist:
        if ','.join(row) in line:
            return True
    return False

def write_new_score(file, line):
    if( not line_is_exist(file, line) ):
        with open(file, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(line)
    else:
        print('line exsist already')
    

In [3]:
data= pd.read_csv('../data.csv')

In [4]:
##copying data
data1=data.copy()
### spliting data en X et Y
X= data1.drop('Loan Status', axis=1)
Y= data1['Loan Status']
### spliting the data to train and test
X_train , X_test , Y_train , Y_test = train_test_split(X, Y, test_size = 0.30, random_state = 123)
## dicision tree (CART)
dct = DecisionTreeClassifier()
params={'criterion' : ['gini', 'entropy'],
        'max_depth': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14, 15, 16, 17, None]}

##accuracy
grid_search_acc= GridSearchCV(estimator=dct, param_grid= params, scoring='accuracy',cv=2, n_jobs=-1)
grid_search_acc= grid_search_acc.fit(X_train, Y_train)
y_predict= grid_search_acc.best_estimator_.predict(X_test)


In [5]:
## get avg precision & avg recall
report= classification_report(Y_test, y_predict, output_dict=True)
avg_list = report.pop("weighted avg")
avg_precision= round(avg_list['precision'], 3)
avg_recall= round(avg_list['recall'], 3)
accuraccy= round(accuracy_score(Y_test,y_predict), 3)
## csv row
csv_row = [get_algo_path(), 'Decision Tree', str(grid_search_acc.best_params_), str(accuraccy), str(avg_precision), str(avg_recall)]
## write file
csv_file= get_csv_path()
write_new_score(csv_file, csv_row)

In [6]:
## evaluating the model
print("----------")
print('best parameters: ', grid_search_acc.best_params_)
print("----------")
print(confusion_matrix(Y_test,y_predict))
print("----------")
print(classification_report(Y_test, y_predict))
print("----------")


----------
best parameters:  {'criterion': 'entropy', 'max_depth': 7}
----------
[[ 1297  3655]
 [    6 13405]]
----------
              precision    recall  f1-score   support

           0       1.00      0.26      0.41      4952
           1       0.79      1.00      0.88     13411

    accuracy                           0.80     18363
   macro avg       0.89      0.63      0.65     18363
weighted avg       0.84      0.80      0.75     18363

----------


In [7]:
grid_search_acc.cv_results_



{'mean_fit_time': array([0.14909983, 0.18500364, 0.20794344, 0.25980556, 0.26080179,
        0.28224385, 0.33460546, 0.34707248, 0.33859468, 0.37549472,
        0.42137265, 0.4303484 , 0.44530821, 0.44780314, 0.55451643,
        0.19148993, 0.24983156, 0.26878011, 0.32463109, 0.33310854,
        0.38546872, 0.44082105, 0.48470366, 0.47672486, 0.5305804 ,
        0.5450418 , 0.59740222, 0.57795405, 0.54404461, 0.53905749]),
 'std_fit_time': array([0.00548506, 0.00748003, 0.01047301, 0.00249374, 0.00149655,
        0.00099695, 0.01446104, 0.01196778, 0.00249267, 0.01446056,
        0.00448704, 0.01745415, 0.01745391, 0.00698149, 0.01196778,
        0.0049839 , 0.02842391, 0.00648224, 0.02643049, 0.00199521,
        0.01146984, 0.01495993, 0.01296532, 0.00099778, 0.01994598,
        0.01845002, 0.00698078, 0.01047134, 0.00249374, 0.01944852]),
 'mean_score_time': array([0.00947559, 0.0099746 , 0.01097143, 0.01246643, 0.01097023,
        0.01097047, 0.01246607, 0.01097071, 0.01097023, 0.01